In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader

# 문서 로드
loader = PyMuPDFLoader('../data/KCI_FI003153549.pdf')
documents = loader.load()

In [5]:
len(documents)

12

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splitted_documents = text_splitter.split_documents(documents)

In [8]:
len(splitted_documents)

42

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# 임베딩 모델 준비
embedding_model = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001",
)

In [16]:
from langchain_community.vectorstores import FAISS

# FAISS 벡터스토어가 존재하는 경우에는 덮어쓰기 하지 않고 로드
FAISS_INDEX_PATH = "faiss_index"

if os.path.exists(FAISS_INDEX_PATH):
    print(f"FAISS 인덱스 {FAISS_INDEX_PATH}를 로드합니다.")
    vectorstore = FAISS.load_local(
        FAISS_INDEX_PATH,
        embedding_model,
        allow_dangerous_deserialization=True,
    )
else:
    print(f"FAISS 인덱스 {FAISS_INDEX_PATH}가 없으므로 생성합니다.")
    
    # FAISS 벡터스토어 생성 및 저장
    vectorstore = FAISS.from_documents(splitted_documents, embedding_model)
    vectorstore.save_local(FAISS_INDEX_PATH)
    print(f"FAISS 인덱스를 {FAISS_INDEX_PATH}에 저장했습니다.")

FAISS 인덱스 faiss_index를 로드합니다.


In [ ]:
# vectorstore = None

In [17]:
vectorstore

In [18]:
retriever = vectorstore.as_retriever()

In [19]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029C8A757C50>, search_kwargs={})

In [20]:
# 예시 질의
query = "본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수와 문서 유형별 비율은 어떻게 되나요?"

In [21]:
retriever.invoke(query)

[Document(id='1e7c1c04-1d78-418d-a4bc-56556251c239', metadata={'producer': 'ezPDF Builder Supreme', 'creator': '', 'creationdate': '2024-12-27T02:09:00+09:00', 'source': '../data/KCI_FI003153549.pdf', 'file_path': '../data/KCI_FI003153549.pdf', 'total_pages': 12, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-12-27T02:09:00+09:00', 'trapped': '', 'modDate': "D:20241227020900+09'00'", 'creationDate': "D:20241227020900+09'00'", 'page': 4}, page_content='의료기기 임상시험 분야의 도메인 특성에 맞게 튜닝하\n기 위해 의료기기 임상시험 전문가로부터 총 158개의 문\n서(총 11,954 페이지)를 수집하였다. 수집된 문서는 다음\n과 같이 분류된다:\ny 규제 문서 (30%): FDA, EMA, PMDA 가이드라인, \nGCP 문서 등\ny 교육 자료 (20%): 임상시험 수행자 교육 매뉴얼, 온라\n인 강의 자료 등\ny 프로토콜 및 보고서 (25%): 임상시험 프로토콜, CSR \n(Clinical Study Report) 템플릿 등\ny 의료기기 특화 문서 (15%): 의료기기 임상시험 계획\n서, 기술문서 등\ny 기타 (10%): 윤리위원회 관련 문서, 환자 동의서 템플\n릿 등\n1.2 Validity of Collected Data\n수집된 \n데이터셋은 \n의료기기 \n임상시험에 \n특화된 \nPrivate LLM 구축을 위해 도메인 적합성과 다양성, 그리\n고 응용 가능성 측면에서 높은 타당성을 갖추고 있다. 

In [22]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    '''다음 컨텍스트만 사용해 질문에 답하세요.
컨텍스트: {context}

질문: {question}
'''
)

### 지식 기반 답변 생성

#### 1. 딕셔너리

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

chain = prompt | llm | StrOutputParser()

In [ ]:
results = retriever.invoke(query) # 벡터DB 검색 결과

In [25]:
chain.invoke({'context': results, 'question': query})

'본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수는 **11,954 페이지**입니다.\n\n문서 유형별 비율은 다음과 같습니다:\n*   **규제 문서**: 30%\n*   **교육 자료**: 20%\n*   **프로토콜 및 보고서**: 25%\n*   **의료기기 특화 문서**: 15%\n*   **기타**: 10%'

#### 2. retriever, RunnablePassthrough

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [27]:
chain.invoke(query)

'본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수는 **111,954페이지**이며, 문서 유형별 비율은 다음과 같습니다.\n\n*   **규제 문서**: 30% (FDA, EMA, PMDA 가이드라인, GCP 문서 등)\n*   **교육 자료**: 20% (임상시험 수행자 교육 매뉴얼, 온라인 강의 자료 등)\n*   **프로토콜 및 보고서**: 25% (임상시험 프로토콜, CSR 템플릿 등)\n*   **의료기기 특화 문서**: 15% (의료기기 임상시험 계획서, 기술문서 등)\n*   **기타**: 10% (윤리위원회 관련 문서, 환자 동의서 템플릿 등)'